# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

**Цель:** обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.Построить модель со значением метрики качества F1 не меньше 0.75.

**План работы:**
1. Загрузить и подготовить данные.
2. Обучить разные модели.
3. Сделайть выводы.

Данные находятся в файле `toxic_comments.csv`.
Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак

## Подготовка

In [1]:
import warnings
import re
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, make_scorer
from transformers import BertTokenizer, BertModel

In [2]:
warnings.filterwarnings('ignore') # скроем лишние предупреждения

In [3]:
df = pd.read_csv('toxic_comments.csv', index_col=[0])

In [4]:
# функция для описания датасета
def df_describe(df):
    display(df.head(10))
    print('Общая информация о полученном датафрейме:')
    df.info()
    print('Описание данных:')
    display(df.describe())
    print('Количество пустых значений:')
    display(df.isna().sum())
    print('Количество явных дубликатов:')
    display(df.duplicated().sum())

In [5]:
df_describe(df)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


Общая информация о полученном датафрейме:
<class 'pandas.core.frame.DataFrame'>
Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB
Описание данных:


,toxic
count,159292.000000
mean,0.101612
std,0.302139
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


Количество пустых значений:


text     0
toxic    0
dtype: int64

Количество явных дубликатов:


np.int64(0)

In [6]:
df['toxic'].value_counts(normalize=True)

toxic
0    0.898388
1    0.101612
Name: proportion, dtype: float64

Количество токсичных комментариев приблизительно равно 10.2%. Наблюдается дисбаланс классов.

Чтобы сократить время обучения, сократим размеры выборок.

In [7]:
# создаим новый датафрейм с 8000 объектами
df_new = df.sample(n=8000, random_state=42)
df_new.shape

(8000, 2)

In [8]:
df_new['toxic'].value_counts(normalize=True)

toxic
0    0.898625
1    0.101375
Name: proportion, dtype: float64

**Вывод:**  данные были загружены. В данных 159292 строки и 2 столбца с текстом и целевым признаком. В исходных данных наблюдается дисбаланс классов, принято решение его сохранить. Размер выборки сокращен до 8000 объектов.

## Обучение

### Подготовка даннных с BERT

In [9]:
# используем предобученную модель BERT, адаптированную для токсичных комментариев (unitary/toxic-bert)
pretrained_weights = 'unitary/toxic-bert'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights, do_lower_case=True)

# инициализируем модель
model = BertModel.from_pretrained(pretrained_weights)

Максимальное количество токенов, которые может обработать BERT равно 512. Чтобы сократить время обработки и используемую память, ограничим количество токенов до 200.

In [10]:
max_len = 200
tokenized = df_new['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_len, truncation=True))

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [11]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/80 [00:00<?, ?it/s]

In [12]:
features = np.concatenate(embeddings)

display(features.shape)

(8000, 768)

**Вывод:** для токенизации и получения векторных представлений текстов использовали предобученную модель BERT, адаптированную для токсичных комментариев (unitary/toxic-bert).

### Обучение моделей

In [13]:
# формируем выборки
X = features
y = df_new['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Логистическая регрессия

In [14]:
# инициализируем модель
model = LogisticRegression(solver='saga', random_state=42)

# гиперпараметры
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

# метрика F1
f1_scorer = make_scorer(f1_score, average='binary')

# перебор параметров на тренировочной выборке
grid_search_lr = GridSearchCV(model, param_grid, cv=5, scoring=f1_scorer)
grid_search_lr.fit(X_train, y_train)

print('Logistic Regression(saga solver)')
print(f'Best parameters: {grid_search_lr.best_params_}')
best_score = grid_search_lr.best_score_
print(f"Best Cross-Validation Score: {best_score:.4f}")

# сохраняем лучшую модель
best_model_log_reg = grid_search_lr.best_estimator_

Logistic Regression(saga solver)
Best parameters: {'C': 0.1, 'penalty': 'l2'}
Best Cross-Validation Score: 0.9579


### RandomForestClassifier

In [15]:
# инициализация модели
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# гиперпараметры
param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [1, 2, 5],
    'min_samples_split': [2, 5, 10],
}

# перебор параметров на тренировочной выборке
grid_search_rf = GridSearchCV(rf_classifier, param_grid, cv=5, scoring=f1_scorer)
grid_search_rf.fit(X_train, y_train)

print('RandomForestClassifier')
print(f'Best parameters: {grid_search_rf.best_params_}')
print(f"Best Cross-Validation Score: {grid_search_rf.best_score_:.4f}")

# сохраняем лучшую модель
best_model_rand_for = grid_search_rf.best_estimator_

RandomForestClassifier
Best parameters: {'max_depth': 2, 'min_samples_split': 2, 'n_estimators': 20}
Best Cross-Validation Score: 0.9501


### HistGradientBoostingClassifier

In [16]:
# инициализируем классификатор XGBoost
hgb_classifier = HistGradientBoostingClassifier(max_iter=100, random_state=42)

# гиперпараметры
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_leaf_nodes': [15, 31, 63],
    'min_samples_leaf': [10, 20, 30],
}

# перебор параметров на тренировочной выборке
grid_search_hgb = GridSearchCV(hgb_classifier, param_grid, cv=5, scoring=f1_scorer)
grid_search_hgb.fit(X_train, y_train)

print('BoostClassifier')
print(f'Best parameters: {grid_search_hgb.best_params_}')
best_score = grid_search_hgb.best_score_
print(f"Best Cross-Validation Score: {best_score:.4f}")

# сохраняем лучшую модель
best_model_hgb = grid_search_hgb.best_estimator_

BoostClassifier
Best parameters: {'learning_rate': 0.2, 'max_leaf_nodes': 63, 'min_samples_leaf': 10}
Best Cross-Validation Score: 0.9507


In [17]:
# смотрим результат работы трех моделей
models = pd.DataFrame({
    'Модель': ['Логистическая регрессия', 'RandomForestClassifier', 'HistGradientBoostingClassifier'],
    'f1': [grid_search_lr.best_score_, grid_search_rf.best_score_, grid_search_hgb.best_score_]
})

models

,Модель,f1
0,Логистическая регрессия,0.957928
1,RandomForestClassifier,0.950106
2,HistGradientBoostingClassifier,0.950706


Таким образом, наилучший результат показала модель логистичекой регресии. Посмотрим результат на тестовой выборке.

### Лучшая модель на тестовой выборке

In [18]:
# прогнозируем на тестовой выборке
y_pred = best_model_log_reg.predict(X_test)

# вычисляем F1
f1_test = f1_score(y_test, y_pred, average='binary')

print(f"F1 Score на тестовой выборке: {f1_test:.4f}")

F1 Score на тестовой выборке: 0.9565


**Вывод:** на тестовых данных модель логистической регрессии показала значение. метрики F1 > 0.75, что соответсвует условиям проекта.

## Выводы

В ходе работы на проектом:
* Данные были загружены. В данных 159292 строки и 2 столбца с текстом и целевым признаком. В исходных данных наблюдался дисбаланс классов, который был исправлен. Размеры выборки сокращено до 8000 объектов.
* Для токенизации и получения векторных представлений текстов использовали предобученную модель BERT, адаптированную для токсичных комментариев (unitary/toxic-bert).
* Обучены модели логистической регрессии, RandomForestClassifier, HistGradientBoostingClassifier
* Наилучший результат метрики F1 на тренировочных данных показала модель логистической регрессии.
* На тестовых данных модель логистической регресии показала значение метрики F1 > 0.75, что соответсвует условиям проекта.